<a href="https://colab.research.google.com/github/mcochranca/Front_Desk/blob/master/Biopython_%7C_Bioinformatics_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'gono-unitigs:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F342174%2F722918%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240530%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240530T173820Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D36d9ed729ec0f210951cea7483458842c5a91b9c1c5eb408bdfa8fe889f04c9dae62b013da898a9bbff87cc1034c48e03374780f1cd1a0e7051dc9423ce64c7a25c9506f01f067760aa23114b5c0d692bd7798889baa88183c1d4ac295a8cccac78ea78ce64ce5d3998c04e575d5a77ec724f3e1e1bc088c143760cd93950ea4b68ef9e72223e890d1f063018c9d6253e1b8a011dbf9aad40179d1d3ce3bb33759364fbdb772b8f9db7fa7a2b2eaf916d2948218aa45ff888de424400117b67ec014901a4628c01baf98da183c3dbcdb5bbc438fff4f9adeaaf2438f9e41d0cf63606ba93c6348a19b363dbc99e63ac7eb325739e7a8c08a0d53c8c5d1a88834,coronavirus-genome-sequence:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F533438%2F976277%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240530%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240530T173820Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2cba122e000a4931649a330f772ec0278c01c3b1d17d5c3c13d69b29b554b017b1603b1e51f390d599b33d4331b96c0624b6614f1ee82b169f3d98058a79b3d28a505204ec410cf345fe27ce737bb1da34180683cafc72a06e61e9ff8f24b43c2e4a0711800cdedcb538eaccbc7a539f433510ddf59608a32ed545487db03bd87cc00c40faade1e2d6abb13c5101fc86e0e8d97df2fbdf3a4cb21f124f28187b8b20d8db7e518eb820f0269781a7244b607cedf4e87531809f3871dbef31dd6d2cc949dceb0ed8c8123e8616be77500a06b7829d6db71acab0f39e0ca33904d73bade1fa91c24a5eeb0ea684eb9b8124319c75d1a40ea26c44bda49abaa36c15,biopython-genbank:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1407797%2F2734138%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240530%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240530T173820Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D595a3bfc7c94a8eb2aae53945e0c08b80339a2d8afaf6ec9d42eb521d27cf6e83d893c67a645b4ea45b89bc2166c86c00e3bf48b27a96975e0491280ca78837bb758f5d896a01b88a3f0cbf96c0e908928d47b206088775663df0267e75db848ca6df64ac15bbc66b628b82c6dc522dada6623d464c9b1947c389d38dc00553e3a8cd27194d52c8ce33c71e5cf7c20f0e080bdf8db9e55aa221c5344d3be9980b04163712c809d5f0a239232bb07707115b19efee4fefca9321185738bc1c72409d3d03dee10931757182261e8be3239fe2c89f84ac90d4cc8d05b5d3ec623581f463d1d0f7b863434c4377d972ddf244cc5dde57989cac17c70d42ee0ffcb5e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import os
import numpy as np
import pandas as pd
from Bio.pairwise2 import format_alignment
from Bio.SubsMat import MatrixInfo
from Bio import pairwise2
from Bio import SeqIO, SearchIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.PhyloXML import Phylogeny
from Bio import Phylo

from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='white')
blast_id = True

![](https://i.imgur.com/lddIujY.jpg)

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>1 |</span></b> <b>INTRODUCTION</b></div>

### <b><span style='color:#E888BB'> 1.1 |</span> Bioinformatics meets Python </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>BIOPYTHON</span></b></p></div>

- The Biopython Project is an international association of developers of freely available Python tools for computational molecular biology.
- Basically, the goal of Biopython is to make it as easy as possible to use Python for bioinformatics by creating high-quality, reusable modules and classes.
- Biopython features include parsers for various Bioinformatics file formats (**<span style='color:#F55AA2'>BLAST</span>**,**<span style='color:#F55AA2'>Clustalw</span>**, **<span style='color:#F55AA2'>FASTA</span>**, **<span style='color:#F55AA2'>Genbank</span>**, access to <b>online services</b> (**<span style='color:#F55AA2'>NCBI</span>**, Expasy,...)
- Interfaces to common and not-so-common programs (Clustalw, DSSP, MSMS...), a standard sequence class, various clustering modules, a KD tree data structure etc. and even documentation.

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>NOTEBOOK CONTENT</span></b></p></div>

- In this notebook, we'll be covering very simple, beginner entry operations one can encounter when starting out in the field of <b><span style='color:#F55AA2'>bioinformatics</span></b>.
- <b><span style='color:#F55AA2'>BioPython</span></b> is a very convenient tool we can utlise when working with <b><span style='color:#F55AA2'>Sequences</span></b> & <b><span style='color:#F55AA2'>Alignments</span></b> of <b><span style='color:#F55AA2'>DNA/Protein</span></b> based sequences.
- Whilst it can be interesting to work with your own classes you can modify to your needs, it's much easier and less of a hastle to work in BioPython.
- It is [well documented](https://biopython.org) & is quite popular, so there are plenty of of [examples](http://diverge.hunter.cuny.edu/~weigang/BioPython.html) that you can come across & will potentially help you understand how you can utilise this tool

### <b><span style='color:#E888BB'> 1.2 |</span> Biological Sequence Alphabet </b>

Two sets of alphabets are used in bioinformatics, **<mark style="background-color:#5D2ECC;color:white;border-radius:5px;opacity:0.7">Nucleotides</mark>** & **<mark style="background-color:#5D2ECC;color:white;border-radius:5px;opacity:0.7">Amino Acids</mark>**

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.7">ABC (I/II)</mark>** **<mark style="background-color:#F55AA2;color:white;border-radius:5px;opacity:0.7">Nucleic Acids</mark>**

> - Among <b><span style='color:#5D2ECC'>molecules with a biological role</span></b>, we can find <b><span style='color:#5D2ECC'>nucleic acids</span></b>.
> - Nucleic acids encode and express the genetic code that is kept within the cell.
- There are two major types of <b><span style='color:#5D2ECC'>nucleic acids</span></b>:
> - <b>DeoxyriboNucleic Acid (DNA)</b>
> - <b>RiboNucleic Acid (RNA)</b> (Obtainable via transcription)
- DNA contains the information necessary to build a cell, and keep it functioning.
- In <b><span style='color:#5D2ECC'>eukaryotic</span></b> cells, DNA will be found in the nucleus, whilst in the <b><span style='color:#5D2ECC'>prokaryotic</span></b> cells, it will be found in the cytoplasm.
- <b><span style='color:#5D2ECC'>IUPAC</span></b> defines the full list of nucleotides as shown in the table below, with <b>A,T,G,C</b> being the main four:
- Another type of nucleotide list often used is **[IUB Ambiguity Codes](http://biocorp.ca/IUB.php)**, which we use later in the notebook as well

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.7">ABC (II/II)</mark>** **<mark style="background-color:#F55AA2;color:white;border-radius:5px;opacity:0.7">Amino Acids</mark>**
- **<span style='color:#5D2ECC'>Amino acids</span>**:
> The **<span style='color:#5D2ECC'>building blocks of proteins</span>**, which are <b>macromolecules</b> that perform most of the functions inside a cell
- Proteins have a **<span style='color:#5D2ECC'>broad range of functions</span>**, spanning from **<span style='color:#5D2ECC'>catalytic</span>** to **<span style='color:#5D2ECC'>structural functions</span>**:

> - **<span style='color:#5D2ECC'>Enzymes</span>** : Type of abundant proteins that promote chemical reactions and convert certain molecules into other types of molecules required for the functioning of the cell.
> - **<span style='color:#5D2ECC'>Carbohydrates</span>** : Serve as energy storage, both for immediate and long term energy demands.
> - **<span style='color:#5D2ECC'>Lipids</span>**: Part of the plasma membrane, doing signaling and energy storage.
- The cell also contains other components of varying complexity. Of importance:
> - <b>Mitochondria</b> & the <b>Chloroplasts</b> : Organelles involved in the production of energy.
> - <b>Ribosomes</b> : Large and complex molecules composed of a mixture of genetic material, req. to assemble proteins and play a central role in the flow of genetic information.

### <b><span style='color:#E888BB'> 1.3 |</span> DNA strands </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>COMPLEMENTARY STRANDS</span></b></p></div>

- DNA is a molecule composed of **<span style='color:#5D2ECC'>two complementary strands</span>** that form and stick together due to the connections established between the nucleotides in both strands.

> - This is made possible by due to the chemical phenomenon where **<span style='color:#5D2ECC'>Adenine (A)</span>** bonds only with **<span style='color:#5D2ECC'>Thymine (T)</span>** nucleotides, as a result of two hydrogen connections.
> - Similarly, **<span style='color:#5D2ECC'>Guanine (G)</span>** bonds only with **<span style='color:#5D2ECC'>Cytosine (C)</span>** nucleotides by three hydrogen connections.

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>REVERSE COMPLEMENT</span></b></p></div>

- This results in **<span style='color:#5D2ECC'>two complementary</span>** and **<span style='color:#5D2ECC'>anti-parallel strands</span>** (connected in opposite directions), if we know the nucleotide sequence in one of the strands, we can get the sequence in the opposite strand by taking the complement of its nucleotides, which are also read backwards, thus we have the **<span style='color:#5D2ECC'>reverse complement</span>** of the other strand.
- It has become a **<span style='color:#5D2ECC'>standard to describe the DNA though only one</span>** of the strands, due to this **<span style='color:#5D2ECC'>complementarity</span>** using <b>[A,T,G,C]</b>.
- The existence of these two strands is essential in order to **<span style='color:#5D2ECC'>pass on genetic information</span>** to new cells and **<span style='color:#5D2ECC'>produce proteins</span>**.


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>2 |</span></b> <b>SEQUENCE DEFINITION</b></div>

### <b><span style='color:#E888BB'> 2.1 |</span> Strings to Sequence </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SMALLER SEQUENCES</span></b></p></div>

- When a sequence are quite short (eg. when we are looking at part of the entire DNA sequence), we can probably just define them using the **<span style='color:#F55AA2'>string</span>** format.
- To use some operations with this sequence in <b>biopython</b> (eg. sequence alignment), we need to <b><span style='color:#F55AA2'>create an instance</span></b> of the <b>Seq</b> class for each sequence.
- In previous biopython versions, we had to specify the <b><span style='color:#F55AA2'>sequence type</span></b>, (eg. <code>IUPAC.protein</code>), however this is not necessary in recent versions.

In [ ]:
n = 'ATGACGGATCAGCCGCAAGCGGAATTGGCGTTTACGTACGATGCGCCGTAA'  # nucleotide sequence
aa = 'MMMELQHQRLMALAGQLQLESLISAAPALSQQAVDQEWSYMDFLEHLLHE' # protein sequence

seq_n = Seq(n)
seq_aa = Seq(aa)

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SIMPLE SEQUENCE OPERATIONS</span></b></p></div>

- Since we don't have <code>.alphabet</code> details, you'll have to know the <b>sequence type</b> you are using, so it makes sense to give them names accordingly.
- An example operation you could apply on a <b>nucleotide sequence</b> would be the **<span style='color:#F55AA2'>reverse_complement</span>**, which is an operation applied to a <b><span style='color:#F55AA2'>DNA</span></b> based sequence.

In [ ]:
print(seq_n.reverse_complement()) # possible
print(seq_aa.reverse_complement()) # not actually possible

### <b><span style='color:#E888BB'> 2.2 |</span> Sequence Annotation Objects </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>DEFINIG MORE DETAIL</span></b></p></div>

- Sequence files obtained from databases provied the option to for more <b>detailed information</b> storage about the sequence, not just the <b>sequence</b> alone.
- In BioPython, we can use <code>SeqRecord</code> objects with our defined <code>Seq</code> object instances & add define additional information about our sequence(s).
- Two commonly added detailed information about a sequence:
  > - <b><span style='color:#F55AA2'>Annotations</span></b> ( Extra metadata information about the sequence )
  > - <b><span style='color:#F55AA2'>Features</span></b> ( Extra information about the sequence content, which is extractable )

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SEQ RECORD CLASS</span></b></p></div>

<b>SeqRecord</b> | [BioPython](https://biopython.org/wiki/SeqRecord)

> SeqRecord objects are used in Biopython to hold a sequence (as a Seq object) with identifiers (ID and name), description and optionally annotation and sub-features.

- <code>SeqRecord</code> is class we'd use when we want to work with <code>Seq</code> that have more information about the sequence, grouped all in one spot.
- If we wanted to do <b>multiple sequence alignment</b>, we have to define each individual sequences in the <code>SeqRecord</code> object, which requires just the <b>id</b> to be defined.

<code>SeqRecord</code> objects can store:
> - <code>seq</code> - The sequence itself ( <code>Seq()</code> objects )
> - <code>id</code> - Sequence identifier
> - <code>name</code> - Sequence name
> - <code>description</code> - Description of the sequence
> - <code>annotations</code> - Global annotations for the whole sequence (<b>dic</b>)
> - <code>features</code> - Structured features (<b>lst</b> of <code>SeqFeature</code> objects)
> - <code>letter_annotations</code> - Annotations for each letter (position) in the sequence
> - <code>dexrefs</code> - References to databases

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SEQ RECORD ANNOTATIONS & SAVING FILES</span></b></p></div>

- As we saw, <code>SeqRecord</code> can hold information about different <b><span style='color:#F55AA2'>annotations</span></b> for the sequence in question.
- Let's define some basic annotations for <code>SeqRec</code> below, which we <b>can subsequently save</b> using <code>SeqIO</code> using the <b><span style='color:#F55AA2'>minimalistic</span></b> <b>FASTA</b> or <b><span style='color:#F55AA2'>detailed</span></b> <b>Genbank</b> formats.
- We'll look at which <code>annotations</code> data are typically added in <b>Section 3</b>, where we'll load and look at <b>real sequence</b> files.

In [ ]:
seq_record = SeqRecord(seq_n)

seq_record.id = "ABC12345"
seq_record.description = 'Neucleotide sequence'
seq_record.annotations['molecule_type'] = 'DNA'
SeqIO.write(seq_record,"/kaggle/working/my_seq.fasta","fasta")
SeqIO.write(seq_record,"/kaggle/working/my_seq.gb","genbank")

In [ ]:
# As we can see FASTA doesn't seem to save the annotations data
print('FASTA Annotations:')
read_seq1 = SeqIO.read('/kaggle/working/my_seq.fasta','fasta')
print(read_seq1.annotations)  # annotations

# Whereas Genbank does
print('GenBank Annotations')
read_seq2 = SeqIO.read('/kaggle/working/my_seq.gb','genbank')
print(read_seq2.annotations)  # annotations

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SEQRECORD FEATURES</span></b></p></div>

- <b>Real sequences</b> can be very long & contain useful <b>subset sequences</b> that one might be interested in.
- <b><span style='color:#F55AA2'>Feature descriptions</span></b> can be quite useful; if we want to define subsets of the <b>sequence</b> to which a <b>feature</b> is applicable.
- More detailed information about the <code>SeqFeature</code> class | [BioPython Documentation](https://biopython.org/docs/1.75/api/Bio.SeqFeature.html)

<b>Some</b> <code>SeqFeature</code> object attributes:
> - <code>location</code> - Indicator of which region the annotation applies ( <code>FeatureLocation</code> object )
> - <code>type</code> - Feature type ( <code>string</code> )
> - <code>qualifiers</code> - Additional Info ( <code>dict</code> )



In [ ]:
from Bio import SeqFeature

# Define locations
s0 = SeqFeature.AfterPosition(10) # exact location
s1 = SeqFeature.BetweenPosition(40,left=35,right=40) # fuzzy location (range)
location = SeqFeature.FeatureLocation(s0,s1) # define the feature location
print(f'Feature Location Notation: {location}')

In [ ]:
# Example for a gene feature

unitig = Seq('GTGCGACAGCAAAGTCCAAACCAGCGTCCCCGCC')
feature = SeqFeature.SeqFeature(SeqFeature.FeatureLocation(5,10), # location (shorter)
                                type='gene', # the specified type of the feature
                                strand=-1  # -1 indicates the minus strand (reverse_complement)
                               )

# If we wanted to extract the feature
feature_seq = feature.extract(unitig)
print(feature_seq)

We can of course add this feature to the <code>SeqRecord</code> class & save, subsequently recalling the feature again.

In [ ]:
seq_record = SeqRecord(unitig)

seq_record.id = "ABC12345"
seq_record.description = 'Neisseria gonorrhoeae unitig'
seq_record.annotations['molecule_type'] = 'DNA'
seq_record.features = [feature]

In [ ]:
# We can of course write to gb file & see our stored data
SeqIO.write(seq_record,"/kaggle/working/my_seq.gb","genbank")
read_seq2 = SeqIO.read('/kaggle/working/my_seq.gb','genbank')
print(read_seq2.features)  # annotations

# define one of the features from the list
feature = read_seq2.features[0]

# Extracted feature
feature_seq = feature.extract(read_seq2)
print('\nExtracted Feature:')
print(feature_seq)

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>3 |</span></b> <b>READING SEQUENCE FILES</b></div>

### <b><span style='color:#E888BB'> 3.1 |</span> Common Formats </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>TWO MAIN FORMATS</span></b></p></div>

- Two formats are commonly used for sequence storage, introduced in the previous section.
- In <b>BioPython</b>, <b><span style='color:#F55AA2'>FASTA</span></b> & <b><span style='color:#F55AA2'>GenBank</span></b> file formats can be read using BioPython's <code>Bio.SeqIO</code> class.

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SIMPLE OR COMPLEX FORMAT</span></b></p></div>

- <b>FASTA</b> is quite a <b>generic format</b> & can be used to store <b>multiple sequences</b> & even <b>alignments</b> as well.
- <b>FASTA</b> is definitely less detailed & very minimalistic when it comes to content, usually only containing a <b>description</b> & <b>sequence</b>.
- <b>GenBank</b> files on the other hand tend to be very detailed when it comes to the <b>sequence description</b>.

### <b><span style='color:#E888BB'> 3.2 |</span> FASTA format </b>

Snipplet from **[FASTA format](https://en.wikipedia.org/wiki/FASTA_format)**:
>In bioinformatics and biochemistry, the FASTA format is a text-based format for representing either <b>nucleotide sequences</b> or <b>amino acid (protein) sequences</b>, in which nucleotides or amino acids are represented using single-letter codes. The format also allows for <b>sequence names</b> and <b>comments</b> to precede the sequences. <br>
> The format originates from the FASTA software package, but has now become a near universal standard in the field of bioinformatics.

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SINGLE SEQUENCE FILES</span></b></p></div>

- The FASTA format can actually contain <b>multiple sequences</b> in one file & we need to read them slightly differently.
- <code>SeqIO.read()</code> is used for reading <b><span style='color:#F55AA2'>single sequence</span></b> files, else we get an error <code>More than one record found in handle</code>.

In [ ]:
''' FASTA formats w/ one sequence '''

single_fasta = '/kaggle/input/biopython-genbank/example.fasta' # nuceotide seq
fasta_n = SeqIO.read(single_fasta,'fasta')
print(f'FASTA Content:')
print(fasta_n)      # Print fasta file information

print(f'\nNumber of Nucleotides: {len(fasta_n)}') # number of nucleotides
seq_fasta = fasta_n.seq # Extract Sequence

print('\nFeatures & Annotations:')
print(fasta_n.features)     # some files contain just the basic details
print(fasta_n.annotations)

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>MULTIPLE SEQUENCE FILES</span></b></p></div>

<code>SeqIO.parse()</code> is used for reading <b><span style='color:#F55AA2'>multiple sequence</span></b> files, which returns <b>an iterator</b>, that can be used to cycle through and get the sequences.

In [ ]:
''' FASTA formats w/ multile sequences '''

multi_fasta = '/kaggle/input/biopython-genbank/NC_005816.faa' # protein seq
iter_multi_fasta_aa = SeqIO.parse(multi_fasta,'fasta') # returns an interator

lst_fasta_aa = []
for seq_aa in iter_multi_fasta_aa:
    lst_fasta_aa.append(seq_aa)

print(f'Number of Sequences read: {len(lst_fasta_aa)}')
print(f'\nFirst Sequence: \n{lst_fasta_aa[0]}')

### <b><span style='color:#E888BB'> 3.3 |</span> GENBANK format </b>
Snipplet from [Bacterial Genoes Bioinformatics](https://www.futurelearn.com/info/courses/bacterial-genomes-bioinformatics/0/steps/47012)

> The Genbank format allows for the storage of information in addition to a DNA/protein sequence. It <b>holds much more information</b> than the FASTA format. Formats similar to Genbank have been developed by ENA (EMBL format) and by DDBJ (DDBJ format).

**COMPATIBILITY WITH DATABASES**

> Primary databases have developed highly structured data file formats that enable the storage of all of these additional data that accompany the otherwise “naked” DNA sequence encoded in a FASTA file. The strict layout is necessary for the file to be compatible with a range of computer programs. Each of the three primary databases have their own sequence file format layout. However, all of them contain almost the same fields and the same information, making them interchangeable.

- Like <b><span style='color:#F55AA2'>FASTA</span></b>, <b><span style='color:#F55AA2'>GenBank</span></b> files can store more than one sequence & are read in exactly the same way as shown above using class <code>SeqIO</code>. (<code>.read</code> & <code>.parse</code>)
- However as indicated above, they tend to contain much more in depth information about the sequence, content of the nature storable in the <code>SeqRecord</code> class object.
- As we can see below, they can store both <b><span style='color:#F55AA2'>Annotations</span></b> & <b><span style='color:#F55AA2'>Features</span></b> ( both of which we looked at in <b>Section 2.2</b> ), which makes this format very useful.

In [ ]:
file_gb = '/kaggle/input/biopython-genbank/example.gb'

''' Read GenBank format '''
# GenBank format are more comprehnsive than fasta.

gb = SeqIO.read(file_gb,'genbank')
seq_gb = gb.seq # store sequence

print('Print Seq (preview)')
print(gb.seq[:40])

print('\nLength of Sequence:')
print(len(gb.seq))

print('\nRecord ID:')
print(gb.id)

print('\nName:')
print(gb.name)

print('\nDescription:')
print(gb.description)

# Annotations
print('\nNumber of Annotations:')
print(len(gb.annotations))

# Features
print('\nNumber of Features:')
print(len(gb.features))

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>ANNOTATION CONTENT</span></b></p></div>

- We can visualise the <b><span style='color:#F55AA2'>annotation</span></b> content (if it is present) using <code>.annotations</code>.
- Being a <b>dictionary</b>, we can get see what sort annotations are present by calling <code>.annotations.keys()</code>.

In [ ]:
# annotations field; dic that provides a number of properties for the seq
print('Annotations dictionary:\n')
print(gb.annotations)

print('\nKeys:')
print(gb.annotations.keys())

In [ ]:
# Getting specific key values
print('\nGet specific parts of the annotation:\n')
print('Taxonomy:\n')
print(gb.annotations['taxonomy'])

print('Source:\n')
print(gb.annotations['source'])

print('Date:\n')
print(gb.annotations['date'])

print('gi number:\n')
print(gb.annotations['gi'])

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SHOWING FEATURE CONTENT</span></b></p></div>

- If there are any <b><span style='color:#F55AA2'>features</span></b> present in the file, we can get the <b>list</b> by calling <code>.features</code> ( What we defined in <b>Section 2.2</b> as well )
- <b>Subsets</b> of the <b>sequence</b> can be extracted <code>.extract()</code> from the The <b>list</b> of <code>SeqFeature</code> objects.

In [ ]:
gb.features[:5] # show only a few features (total=41)

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>EXPLORING THE GENE FEATURE TYPE</span></b></p></div>


- One of the features <code>.type</code> that is used in <code>.features</code> is <b><span style='color:#F55AA2'>"gene"</span></b>.
- We can for example, cycling through all features & get all the annotated <b>genes</b> in the whole sequence.
- Exploring these genes, we can get their <b><span style='color:#F55AA2'>locus tag</span></b> (from <b>qualifiers</b>), <b><span style='color:#F55AA2'>database reference</span></b> (from <b>qualifiers</b>),  <b><span style='color:#F55AA2'>strand</span></b> & <b><span style='color:#F55AA2'>location</span></b>.
- We can apply an operation called <b>translate</b> (<b>Section 4.3</b>) to try to <b>find the proteins</b> that the gene feature encode

In [ ]:
gene_features = []
for i in range(len(gb.features)):
    if(gb.features[i].type == 'gene'):
        gene_features.append(gb.features[i])

print(f'Number of gene features: {len(gene_features)}')
gene_features

In [ ]:
# Gene Qualifiers
gene_features[0].qualifiers

In [ ]:
for feat in gene_features:
    print(f"locus tag: {feat.qualifiers['locus_tag']}, database ref: {feat.qualifiers['db_xref']}, strand: {feat.strand}, location: {feat.location}")

In [ ]:
protein_feat = []
for feat in gene_features:
    protein_feat.append(feat.extract(gb.seq).translate(table='Bacterial',cds=True))

protein_feat # list of protein sequences

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>EXPLORING CDS FEATURE TYPE</span></b></p></div>


- Another <code>.type</code> that is used in <code>.features</code> is <b><span style='color:#F55AA2'>"CDS"</span></b> (<b>Coding Sequence</b>)
- <b>CDS</b> <code>qualifiers</code> contain quite a bit more content, including the <b>translation</b> sequence, so we can compare our translation of the <b>gene</b> feature, since both types are reffering to the same parts of the sequence.

In [ ]:
CDS_features = []
for i in range(len(gb.features)):
    if(gb.features[i].type == 'CDS'):
        CDS_features.append(gb.features[i])

print(f"Number of CDS features: {len(CDS_features)}")
CDS_features

In [ ]:
print(f'CDS Qualifier Keys: {CDS_features[0].qualifiers.keys()}\n')

print('Showing First CDS Feature')
pprint(CDS_features[0].qualifiers) # ordered dictionary

In [ ]:
for key, value in CDS_features[0].qualifiers.items():
    print(f'{key} : {value}')

In [ ]:
# Should be identical
Seq(CDS_features[0].qualifiers['translation'][0]) == protein_feat[0]

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>4 |</span></b> <b>SEQUENCE PROCESSING</b></div>

### <b><span style='color:#E888BB'> 4.1 |</span> Basic Seqence Processing </b>

Biopython uses Python <b><span style='color:#F55AA2'>strings</span></b>, so the standard string operations can be used.

In [ ]:
''' Basic Operations '''

print(seq_fasta.count('G'))   # Counter for Guanine Nucleotide Count
print(seq_fasta[1:3])         # Show specific Subset of a Sequence
print('ATA' in seq_fasta)     # Is string subset part of sequence
print(seq_fasta.find('ATA'))  # Find Substring index

# Instantiate a sequence w/ Seq()
seq1 = Seq('MEVRNAKS')
seq2 = Seq('GHERWKY')

print(seq1 + seq2)


### <b><span style='color:#E888BB'> 4.2 |</span> Transcription & Reverse Complement Strand </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>TRANSCRIPTION</span></b></p></div>


The process of <b><span style='color:#F55AA2'>transcription from DNA</span></b> to <b><span style='color:#F55AA2'>RNA</span></b>, is a critical step in the process of protein synthesis:

(I) Process that occurs within the nucleus of a cell; we obtain a __Mature messenger RNA, mRNA.__ <br>
(II) mRNA is then transfered to the cytoplasm, where it will be used by a cellular machine, to guide the production of a protein.
- Given a DNA sequence, transcription is the first step in <b>obtaining a Protein</b>.
- Nucleotide sequence of a gene from one of the DNA strands is transcribed ( copied into a compelentary molecule of RNA )
- Complementarity of code, allows us to recover information contained in the DNA sequence, process performed by an enzyme, RNA polymerase.
- Stabilisation of elements at the end of a molecule are perfomed by different protein complexes.

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>REVERSE COMPLEMENT</span></b></p></div>

- DNA has <b>two complementary strands</b>.
- Due to the complementarity of the DNA strands, usually only one of the strands is provided in a sequence file obtained from databases.
- The second strand to the input <b>DNA sequence</b> can be obtained by calling the <code>.reverse_complement()</code> function.

In [ ]:
''' Transcription & Reverse Complement '''

print(f'DNA Sequence: {seq_fasta[:100]}')
print(f'Reverse Complement: {seq_fasta.reverse_complement()[:100]}')
print(f'Transcription: {seq_gb.transcribe()[:100]}')

### <b><span style='color:#E888BB'> 4.3 |</span> Translation </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>TRANSLATION & CODON TABLE</span></b></p></div>


#### **TRANSLATION**

Snipplet from [genome.gov](https://www.genome.gov/genetics-glossary/Translation)

> Translation is the process of translating the sequence of a <b><span style='color:#F55AA2'>messenger RNA</span></b> (mRNA) molecule to a <b><span style='color:#F55AA2'>sequence of amino acids</span></b> during protein synthesis. The <b><span style='color:#F55AA2'>genetic code</span></b> describes the relationship between the sequence of base pairs in a gene and the corresponding amino acid sequence that it encodes. In the cell cytoplasm, the ribosome reads the sequence of the mRNA in groups of three bases to assemble the protein.

- Translation accepts a DNA sequence and converts it into a <b><span style='color:#F55AA2'>proteins</span></b> in the reading frame you specify.

#### **CODON TABLES**

A <b><span style='color:#F55AA2'>codon</span></b> is a sequence of <b><span style='color:#F55AA2'>three DNA/RNA nucleotides</span></b> that corresponds with a <b><span style='color:#F55AA2'>specific amino acid</span></b> or <b><span style='color:#F55AA2'>stop signal/codon(*)</span></b> during protein synthesis

Snipplet from [Khan Academy](https://www.khanacademy.org/science/ap-biology/gene-expression-and-regulation/translation/a/the-genetic-code-discovery-and-properties)

> The full set of relationships between <b><span style='color:#F55AA2'>codons</span></b> and amino acids (or stop signals) is called the <b><span style='color:#F55AA2'>genetic code</span></b>. The genetic code is often summarized in a table.

There are a number of <b><span style='color:#F55AA2'>codon tables</span></b>, a few mentioned below:
> - Standard  (SGC)
> - Vertebrate Mitochondrial
> - Bacterial

In [ ]:
from Bio.Data import CodonTable

# Let's take a look at one type of table (SGC)
table = CodonTable.unambiguous_dna_by_name['Standard']
print(table)

In [ ]:
# Getting a specific conversion
table.forward_table["CTA"]

In [ ]:
''' Translation for both DNA & RNA '''
#  translate codon into an aminoacid

rna_seq = Seq('AUGCGUUUAACU')
print(f'translation RNA: {rna_seq.translate()}')

# Translate DNA sequence using two different tables (SGC & Vertebrate Mitochondrial)
print(f'translation DNA: {seq_gb.translate()[:100]}')
print(f'translation DNA (alternative): {seq_gb.translate(table="Vertebrate Mitochondrial")[:100]}')

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>PROTEIN IDENTIFICATION</span></b></p></div>


- We can of course load something more realistic, an entire <b>genome</b> sequence as well & apply translation as we try to understand what proteins are encoded in the genome.
- Let's obtain the largest <b>amino acid</b> chain in the sequence data.

In [ ]:
# Load our sequence & apply translate
virus_fna = '/kaggle/input/coronavirus-genome-sequence/MN908947.fna'
virus_SeqRec = SeqIO.read(virus_fna,'fasta')
virus_Seq = virus_SeqRec.seq

translation = virus_SeqRec.translate()
print(f'Translation length: {len(translation)}')

# functional proteins are chains above 20 amino acids.

# Get Proteins only
proteins = []
for i in translation.seq.split('*'):
    if(len(i) > 19):
        proteins.append(str(i))

df = pd.DataFrame({'protein_chains':proteins})
df['length'] = df['protein_chains'].apply(len)
df = df.sort_values(by = ['length'],ascending=False) [:10]
display(df.head())

# select one protein
one_large_protein=df.nlargest(1,'length')
single_prot = one_large_protein.iloc[0,0]

# write to file which we'll recall later
with open("/kaggle/working/protein.fasta","w") as file:
    file.write(">large protein\n" + single_prot)

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>5 |</span></b> <b>PAIRWISE SEQUENCE ALIGNMENT</b></div>

### <b><span style='color:#E888BB'> 5.1 |</span> Sequence Similarity </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>GENERATING A HYPOTHESIS</span></b></p></div>


- We can generate a hypothesis about the biological function based on <b><span style='color:#F55AA2'>sequence similarity</span></b>.
- If two sequences show high degree of <b><span style='color:#F55AA2'>similarity (score)</span></b>, then they have a high chance of being __homologous__, sharing similar functions.

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SEQUENCE COMPARISON ISSUES</span></b></p></div>


If we try to compare sequences one by one, we might run into issues since:

- Two similar sequences often have gone through  <b><span style='color:#F55AA2'>biological mutation</span></b>, which can change nucleotides, thus changing their structure.
- Thus we cannot simply rely on a comparison of nucleotides that are __in the same order__.
- This process is even more complicated by nucleotide  <b><span style='color:#F55AA2'>insertions</span></b> &  <b><span style='color:#F55AA2'>deletions</span></b>.

As a result, we need to resort to a procedure known as <b><span style='color:#F55AA2'>Pairwise Sequence Alignment (PSA)</span></b>.

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>TYPES OF SEQUENCE ALIGNMENT</span></b></p></div>


There are two main groups of Sequence Alignment:
- <b><span style='color:#F55AA2'>Global sequence alignment</span></b> (we are interested in aligning the entire sequences)
- <b><span style='color:#F55AA2'>Local sequence alignment</span></b> (we aim to find good alignments of adjacent nucleotides of both of the sequences, whilst ignoring the rest)

### <b><span style='color:#E888BB'> 5.2 |</span> Biopython specifics </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>CHOOSING THE TYPE OF ALIGNMENT</span></b></p></div>


- <code>Bio.pairwise2</code> has implementations of <b><span style='color:#F55AA2'>Dynamic Programing (DP)</span></b> Algorithms for __Pairwise Sequence Alignment__.
- Alignment Type option is written after <code>pairwise2.align.X</code>, where <code>.X</code> is replaced by the type; <code>.global</code> and <code>.local</code>.
- After <code>.local</code> or <code>.global</code>, two letters are written to indicate which parameters it takes; eg. <code>.localXY</code>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>OPTIONS FOR X & Y</span></b></p></div>

- The <b><span style='color:#F55AA2'>first (X)</span></b> indicates __a substitution matrix__ or __the parameters for match/missmatches__
- The <b><span style='color:#F55AA2'>second (Y)</span></b> indicates the parameters for the penalty of the gap.

#### **THE SPECIFICS**

Some more specifics<b><span style='color:#F55AA2'> (first letter)</span></b>:
> - If __x__ is provided, a <b>score of 1</b> is considered for the alignment & a <b>mismatch score of 0</b>. <br>
> - If __m__ is provided, the function will allow us to <b>define a match/mismatch score</b> using appropriate parameters. <br>
> - If __d__ is provided, we can pass on a **dictionary** to the function, **defining a full substitution matrix**. <br>

Some more specifics <b><span style='color:#F55AA2'>(second letter)</span></b>:
> - If __x__ is provided, no gap penalties are imposed (g=0) <br>
> - If __s__ is provided, we can define an affinite gap penalty model <br>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>INPUTS FOR SEQUENCES</span></b></p></div>

- Inputs can be both <b>string</b> formats or defined <b>Seq</b> class instances.

### <b><span style='color:#E888BB'> 5.3 |</span> Pairwise Sequence Alignment </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>GLOBAL NUCLEOTIDE SEQUENCE ALIGNMENT</span></b></p></div>


- Let's try an an example, aligning <code>seq1</code> & <code>seq2</code>, both are <b>nucleotide</b> sequences.
- We are looking at <b>global alignment</b> (.global), with a match <b>score of 1</b> & <b>missmatch score of 0</b> (so first letter is x) & <b>no gap penalty emposed</b> (second letter is x).

In [ ]:
''' Global Alignment of two DNA sequences'''

# String based sequences
seq1 = 'ATGGCAGATAGA'
seq2 = 'ATAGAGAATAG'

# define an instance of Seq
cseq1 = Seq(seq1)
cseq2 = Seq(seq2)

# - Global Alignment Problem
# - (match score = 1, missmatch = 0), gap penalties = 0
GDNA = pairwise2.align.globalxx(cseq1,cseq2)
print (f'# alternative optimal alignments: {len(GDNA)}')

# print all the alignments
for i in GDNA:
    print(format_alignment(*i))

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>GLOBAL PROTEIN SEQUENCE ALIGNMENT</span></b></p></div>

- Let's try another example, aligning <code>pseq1</code> & <code>pseq2</code>, both are <b>protein</b> sequences.
- We are looking at <b>global alignment</b> (.global), using the <b>BLOSUM62</b> match/missmatch scores (first letter d) & a <b>gap penalty</b> emposed (second letter s)

In [ ]:
''' Global Alignment of two protein sequences '''
# - Global Alignment Problem of Protein Sequences
# - Using a substitute matrix BLOSUM64, opening gap penalty -4, extension penalty -1

pseq1 = "EVSAW"
pseq2 = "KEVLA"

sm = MatrixInfo.blosum62 # load BLOSUM62 Substitution Matrix
lPRT = pairwise2.align.globalds(pseq1,pseq2,sm,-4,-1)
for i in lPRT:
    print(format_alignment(*i))

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>LOCAL NUCLEOTIDE SEQUENCE ALIGNMENT</span></b></p></div>

- Let's try another example, aligning nucleotide sequences <code>seq1</code> & <code>seq2</code>, this time <b>locally</b>.
- Local alignment (.local), <b>match/missmatch</b> score (first letter m), gap penalty emposed (second letter s)

In [ ]:
''' Local Alignment of two DNA sequences '''
# Local Alignment Problem of DNA Sequences
# Match score = 3, mismatch score = -2, constant gap penalty g=-3 x2

lDNA = pairwise2.align.localms(seq1,seq2, 3,-2,-3,-3)
for i in lDNA:
    print(format_alignment(*i))

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>LOCAL PROTEIN SEQUENCE ALIGNMENT</span></b></p></div>

- Again aligning <code>pseq1</code> & <code>pseq2</code>, both are <b>protein</b> sequences, <b>locally</b>.
- We are looking at <b>local alignment</b> (.local), using the <b>BLOSUM62</b> match/missmatch scores (first letter d) & a <b>gap penalty</b> emposed (second letter s).

In [ ]:
''' Local Alignment of Protein Sequences '''
# - Local Alignment Problem of Protein Sequences
# - Using a substitute matrix BLOSUM64, opening gap penalty -4, extension penalty -1

pPROT = pairwise2.align.localds(pseq1,pseq2,sm,-4,-1)
for i in pPROT:
    print(format_alignment(*i))

### <b><span style='color:#E888BB'> 5.4 |</span> Unitig Sequence Alignment </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>FINDING THE BEST ALIGNMENT IN LIST</span></b></p></div>

- We can utilise <code>pairwise2.align</code> in a variety of situations, eg. cycle through a list & find the best alignment case.
- Having two lists of filled with <b>unitigs</b> (nucleotide sequences); <code>lst1</code> & <code>lst2</code> ( you can also change the input to a sequence in <code>get_bestalign</code>)
- Using the [gonorrhoea](https://www.kaggle.com/nwheeler443/gono-unitigs) dataset, we can compare <b>a sequence from one list</b> and try to find the the best alignment in the second list.
- In this case, we can see that the alignment is not ideal at all, resulting in lots of gaps for the input sequence & the best alignment score is due to the longer matching sequence size.

In [ ]:
# class to used define feature matrix & target variavle
class data:

    def __init__(self):
        self.df = pd.read_csv('../input/gono-unitigs/metadata.csv', index_col=0)
        self.features = self.df.columns
        self.targets = ['azm_sr','cip_sr','cro_sr','cfx_sr','tet_sr','pen_sr']

    def get_case(self,phenotype=None):

        _metadata = self.df
        _metadata = _metadata.dropna(subset=[phenotype])
        _metadata = _metadata[phenotype] # choose target variable

        prefix = '../input/gono-unitigs/'
        suffix = '_gwas_filtered_unitigs.Rtab'

        # store all file pathways
        self.lst_files = []
        for dirname, _, filenames in os.walk('/kaggle/input'):
            for filename in filenames:
                self.lst_files.append(os.path.join(dirname, filename))

        # unitig feature matrix for phenotype
        tdf = pd.read_csv(prefix + phenotype + suffix, sep=" ", index_col=0, low_memory=False)
        tdf = tdf.T # align column data w/ metadata df (pattern_id = sample_idd)
        tdf = tdf[tdf.index.isin(_metadata.index)] # keep only common rows, ie. that have resistence measure
        self.train = tdf
        self.target = _metadata[_metadata.index.isin(tdf.index)]

In [ ]:
gono = data()

# list of nucleotides
gono.get_case('azm_sr')
azm_unitigs = gono.train.columns.tolist() # list of unitigs

print('AZM unitigs:')
print(azm_unitigs[:3])

# List of nucleotides
gono.get_case('cfx_sr')
cfx_unitigs = gono.train.columns.tolist() # list of unitigs

print('\nCFX unitigs:')
print(cfx_unitigs[:3])

In [ ]:
# Select one unitig in lst1 and find the best alignment in lst2
def get_bestalign(lst1,ids,lst2):

    # remove cases if columns exist in string

    lst2_upd = []
    for unitig in lst2:
        if(',' not in unitig):
            lst2_upd.append(unitig)

    # main unitig used for comparison
    n1 = lst1[ids]
    print('Finding Best Alignment Score:')
    print(f'For Sequence: {n1}\n')

    # Cycle through all unitigs in updated list

    max_score = 0
    best_ali = None
    ii=-1
    for unitig in lst2_upd:

        ii+=1;
        ALI = pairwise2.align.localxx(n1,unitig)
        lscore = ALI[0].score
        if(lscore > max_score):
            print(ii,unitig,lscore)
            max_score = lscore
            best_ali = ALI

    return best_ali

best_lalign = get_bestalign(azm_unitigs,10,cfx_unitigs)

In [ ]:
print(format_alignment(*best_lalign[0]))

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>6 |</span></b> <b>MULTIPLE SEQUENCE ALIGNMENT</b></div>

### <b><span style='color:#E888BB'> 6.1 |</span> SeqRecord </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>FOR ALIGNMENT SEQUENCES</span></b></p></div>


- We looked at <code>SeqRecord</code> earlier, a sequence read from a file is already in this format so we can instantly pass them to the alignment class.
- If we define sequences from <b>strings</b> we'll need to define a <code>Seq</code> instance & subsequently <code>SeqRecord</code> (only needing <code>id</code>)
- To align <b><span style='color:#F55AA2'>Multiple Sequences</span></b> together ( <b>More than two</b> ), we can use the <code>.Align</code> class & call <code>MultipleSeqAlignment</code>
- However unlike <code>Bio.pairwise2</code>, we can't use direct <b>strings</b> in the alignment class input & need to define a <code>Seq</code> object for a particular sequence, followed by a defined <b>SeqRecord</b> instance that has its defined <b>sequence ID</b>.

In [ ]:
from Bio.SeqRecord import SeqRecord

# String format of alignment
aa1 = "MHQAIFIYQIGYPLKSGYIQSIRSPEYDNW"
aa2 = "MH--IFIYQIGYALKSGYIQSIRSPEY-NW"
aa3 = "MHQAIFI-QIGYALKSGY-QSIRSPEYDNW"

# Sequence Class instances
seq_aa1 = Seq(aa1)
seq_aa2 = Seq(aa2)
seq_aa3 = Seq(aa3)

# Create a sequence record w/ a defined ID
seqr1 = SeqRecord(seq_aa1,id="seq1")
seqr2 = SeqRecord(seq_aa2,id="seq2")
seqr3 = SeqRecord(seq_aa3,id="seq3")

### <b><span style='color:#E888BB'> 6.2 |</span> MultipleSeqAligmnemt </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>PUTTING IT TOGETHER</span></b></p></div>


- Upon defining the sequence records of all the sequences we want to align, we can call <b><span style='color:#F55AA2'>MultipleSeqAlignment</span></b> with the records as inputs.
- We can also do simple operations on these sequences as they are grouped together, as they are essentially <b>strings</b> in alignment.

In [ ]:
from Bio.Align import MultipleSeqAlignment as MSA

# Multiple Sequence Alignment
alin = MSA([seqr1, seqr2, seqr3])
sub_alin = alin[0:2,:]

In [ ]:
print(alin[1]) # Get the 2nd sequence in the alignment
print(alin[:,2]) # Get the 3rd column in the alignment
print(alin[:,3:7])  # Get multiple columns in alignment
print(alin[0].seq[:3]) # first 3 columns of seq1
print(alin[1:3,5:12]) # sequences 2 and 3; 4th to 10th column


### <b><span style='color:#E888BB'> 6.3 |</span> Alignment Formats </b>


<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>DEFINING AN ALIGNMENT</span></b></p></div>


- Usually alignments are generated by <b><span style='color:#F55AA2'>MultipleSeqAlignment</span></b>, however we can create a blank alignment by passing a blank list to <code>MSA</code>.
- We can also add new sequences by using <code>.add_sequence</code>.

In [ ]:
n1 = 'ACCGGTCAGGGCGACGGGTTTGCGCCCTTTGAT'
n2 = 'ACCGGTCATTTCGACGGGTTTGCGGCCTTTGCT'
n3 = 'CCAGGTCAGCTCGACGGGTTGGCGGCCTTTGCT'

align_n = MSA([])
align_n.add_sequence('n1',n1)
align_n.add_sequence('n2',n2)
align_n.add_sequence('n3',n3)
print(align_n)

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>READING ALIGNMENT DATA</span></b></p></div>


- As with the previously mentioned <b>FASTA</b> & <b>GENBANK</b> formats in <code>SeqIO</code>. There are formats used for <b><span style='color:#F55AA2'>saving</span></b> & <b><span style='color:#F55AA2'>loading alignments</span></b> as well.
- As indicated in the <code>AlignIO</code> [section](https://biopython.org/wiki/AlignIO), <b><span style='color:#F55AA2'>clustal</span></b> is one of such formats & is often used.

In [ ]:
from Bio import AlignIO
alin2 = AlignIO.read("/kaggle/input/biopython-genbank/PF05371_seed.aln", "clustal")


<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>DISPLAY SEQRECORD DATA</span></b></p></div>


Once the <b>clustal</b> format is read, we are dealing with <b>SeqRecord</b> data as we defined earlier.

In [ ]:
# Get the length of the alignment (column size)
print("Size (length):", alin2.get_alignment_length())

# Print the SeqRecord data
for record in alin2:
    print(f'Sequence: {record.seq}, ID: {record.id}')

In [ ]:
# adjust alignment content & write if need be
sub_alin2 = alin2[0:4,:]
AlignIO.write(alin2, "/kaggle/working/sub_alin2.fasta", "fasta")

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>CONVERTING FORMATS</span></b></p></div>

<b>FASTA</b> is quite a generic format, which aside from storing sequence data (which we saw above), can also store alignment data.

In [ ]:
# Write the read clustal formal
AlignIO.write(alin2, "example_alin.fasta", "fasta")

# Or directly convert it
AlignIO.convert("/kaggle/input/biopython-genbank/PF05371_seed.aln", "clustal", "example_alin.fasta", "fasta")

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>VISUALISING ALIGNMENTS</span></b></p></div>

- It can be useful to <b><span style='color:#F55AA2'>colour code</span></b> our alignment data, so we can observe any variation in our sequence more easily.
- We can use the <b>Bokeh</b> library to plot the sequence data using <b>gridplots</b> ( thanks to [Understanding phylogenetic trees](https://www.kaggle.com/nwheeler443/understanding-phylogenetic-trees/notebook) )

In [ ]:
import panel as pn
import panel.widgets as pnw
pn.extension()
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot

def view_alignment(aln, fontsize="9pt", plot_width=800):
    """Bokeh sequence alignment view"""

    #make sequence and id lists from the aln object
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)
    N = len(seqs[0])
    S = len(seqs)
    width = .4

    x = np.arange(0.5,N+0.5)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')
    if N>100:
        viewlen=100
    else:
        viewlen=N
    #view_range is for the close up view
    view_range = (0,viewlen)
    tools="xpan, xwheel_zoom, reset, save"

    #sequence text view with ability to scroll along x axis
    p = figure(title=None, plot_width=plot_width, plot_height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p.add_glyph(source, glyph)
    p.add_glyph(source, rects)

    p.grid.visible = False
    p.xaxis.major_label_text_font_style = "bold"
    p.yaxis.minor_tick_line_width = 0
    p.yaxis.major_tick_line_width = 0

    p = gridplot([[p]], toolbar_location='below')
    return p

# Mapping Function for nucleotide sequences
def get_colors(seqs):
    """make colors for bases in sequence"""
    text = [i for s in list(seqs) for i in s]
    clrs =  {'A':'red','T':'green','G':'orange','C':'blue','-':'white'}
    colors = [clrs[i] for i in text]
    return colors

In [ ]:
p = view_alignment(align_n, fontsize="9pt", plot_width=800)
pn.pane.Bokeh(p)

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>7 |</span></b> <b>BLAST</b></div>

### <b><span style='color:#E888BB'> 7.1 |</span> NCBI Queries </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>USING DATABASES TO UNDERSTAND OUR SEQUENCE</span></b></p></div>

- There are are ways to do <b>Sequence Alignment</b> via websites like [ebi.ac.uk](https://www.ebi.ac.uk/Tools/sss/), which uses an approach called <b><span style='color:#F55AA2'>BLAST</span></b>.
- <b><span style='color:#F55AA2'>Basic Local Alignment Search Tool (BLAST)</span></b> uses the same alignment methods as implemented in <b>Biopython</b>, except the comparison sequences are called from a database.
- BioPython also includes a <b>BLAST</b> module that can be used to search for similar sequences, like we did in <b>Section 4</b> & <b>5</b>; <code>Bio.Blast</code>.
- The alignment requests are <b>made remotely</b>, and it can involve waiting time as our request is processed, however using BLAST locally is also an option in Biopython.


<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>BIOPYTHON BLAST CLASSES</span></b></p></div>

- <code>NCBIWWW</code> : Code to invoke the NCBI BLAST server over the internet (uses [nih.gov](https://blast.ncbi.nlm.nih.gov/Blast.cgi))
- <code>NCBIXML</code> : Code to work with the BLAST XML output. (we need to interpret the received results)


<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SENDING A QUERY</span></b></p></div>

- <code>NCBIWWW.qblast</code> is used to send a request to NCBI BLAST.
- The main parameters used with the <code>NCBIWWW.qblast function</code>: | ([All parameter inputs](https://biopython.org/docs/1.75/api/Bio.Blast.NCBIWWW.html))
> - The function takes in the <code>program option</code> (<b><span style='color:#F55AA2'>"blastn"</span></b>,<b><span style='color:#F55AA2'>"blastp"</span></b>,<b><span style='color:#F55AA2'>"blastx"</span></b>,<b><span style='color:#F55AA2'>"tblastn"</span></b>,<b><span style='color:#F55AA2'>"tblastx"</span></b>)
> - The <code>database</code> to search (eg. <b><span style='color:#F55AA2'>"nr"</span></b>,<b><span style='color:#F55AA2'>"nt"</span></b>, <b><span style='color:#F55AA2'>"pdb"</span></b>)
> - The <code>sequence</code> to be used in the query ( can search by the <b>gi identifier</b> if you have it )

### <b><span style='color:#E888BB'> 7.2 |</span> Python Strings to Sequences </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>LOADING A SEQUENCE</span></b></p></div>

- We'll use a <b>FASTA</b> file whos description contains the actual sequence identification, so we'll be able to see if we have a hit in the database of existing sequences.
- The actual description is not used in the alignment process, it's only there for us to visually confirm we have a match in the database.

In [ ]:
# File which we want to align
# Description is included so we can compare results
file_fasta = '/kaggle/input/biopython-genbank/NC_005816.fna'

record = SeqIO.read(open(file_fasta),format='fasta')
print(record)
print(f'Length of Nucleotide Sequence: {len(record)}')

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>BLAST QUERY AND SAVING CONTENT</span></b></p></div>


- Our sequence (let's say we didn't know the NBCI <b>gi</b> identifier) is made up of <b>nucleotide</b> content, so we'll use <code>'blastn'</code> option when calling <code>.qblast()</code>
- The query can be <b><span style='color:#F55AA2'>saved to a file</span></b> as we would with a normal python file. The content is saved using <code>.read()</code> and saved with the <b>xml</b> extension.

In [ ]:
if(blast_id):

    # BLAST query instance
    result_handle = NCBIWWW.qblast('blastn','nt',record.format('fasta'))

    # Read the file (call only once)
    with open("/kaggle/working/NC_005816_blast.xml", "w") as save_to:
        save_to.write(result_handle.read())
        result_handle.close()

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>READING BLAST SEARCH RESULT CONTENT</span></b></p></div>


- When we want to read the <b>BLAST</b> search results, we can open it as we would any other file with python's <code>open()</code>.
- We need to <b>parse</b> the stored file (<code>NCBIXML.parse</code>), which returns the <code>Record</code> objects, which we can read through using <code>for</code> loops.

In [ ]:
if(blast_id):
    result_handle = open('/kaggle/input/biopython-genbank/NC_005816_blast.xml','r')
    blast_records = NCBIXML.parse(result_handle)

In [ ]:
if(blast_id):

    e_thresh = 0.00; # display only perfect matches
    len_id = 5 # show only 5 results
    for blast_record in blast_records: # go through all blast records

        jj=-1
        for alignment in blast_record.alignments:
            jj+=1;
            if(jj < len_id):
                print(f'\nAlignment #{jj}')
                for hsp in alignment.hsps:
                    if(hsp.expect <= e_thresh):
                        print('***** Alignment *****')
                        print(f'sequence: {alignment.title}')
                        print(f'length: {alignment.length}')
                        print(f'e value: {hsp.expect}')

### <b><span style='color:#E888BB'> 7.3 |</span> BLAST Protein Sequence Query </b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>READING BLAST SEARCH RESULT CONTENT</span></b></p></div>

- In <b>Section 4.3</b>, we identified the longest <b>amino acid</b> chain in the genome in question, let's search for it using <b>BLAST</b>.
- Instead of using <code>NCBIXML.parse</code>, we can also use <code>SearchIO</code> to read our <b>BLAST XML</b> query results.
- For <b>Proteins</b>, we'll need to use <code>"blastp"</code> in our <code>NCBIWWW.qblast</code> query & search in the <code>pdb</code> database.

In [ ]:
def read_blast(result_handle,hit_id=0,show_top=5):

    blast_qresult = SearchIO.read(result_handle,"blast-xml")
    print(blast_qresult[0:show_top])

    #fetch the id, description, evalue, bitscore & alignment
    print(f'Showing BLAST query result #{hit_id}')
    seqid = blast_qresult[hit_id]
    details = seqid[hit_id]

    print(f'\nSequence ID: {seqid.id}')
    print(f'Description: {seqid.description}')
    print(f'e-value: {details.evalue}')
    print(f'Bit Score: {details.bitscore}')

    print('\nShowing Alignment:')
    print(f"\nalignment:\n{details.aln}")

In [ ]:
# read file
if(blast_id):
    c19_protein = SeqIO.read("/kaggle/working/protein.fasta", "fasta")
    result_handle = NCBIWWW.qblast("blastp","pdb",c19_protein.seq)
    read_blast(result_handle) # read qblast result & show first hit only

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>8 |</span></b> <b>PHYLOGENETIC ANALYSIS</b></div>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>SEQUENCE EVOLUTION</span></b></p></div>


- One of the more practical modules applications in BioPython, <code>Bio.Phylo</code> allows us to understand the relation between difference <b>aligned sequences</b>
- **<span style='color:#F55AA2'>Phylogenetic analysis analysis</span>** - concerned with determining how a given <b>set of sequences</b> (alignment) has evolved from a <b>common ancestor</b> through the process of natural evolution.
- The process of evolution is depicted in the form of **<span style='color:#F55AA2'>an evolutionary tree</span>** & bifurcations represent <b>events of mutation</b> from a common ancestor, that gives rise to <b>branches</b>.
- A **<span style='color:#F55AA2'>Phylogenetic tree</span>** will represent known sequences in its leaves & internal tree nodes will represent common ancestors of the sequence below it.

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>USEFUL INFORMATION</span></b></p></div>


- [<b>www <span style='color:#F55AA2'> // </span> PHYLOGENETIC TREES LECTURE</b>](https://www.ncbi.nlm.nih.gov/CBBresearch/Przytycka/download/lectures/PCB_Lect11_Phylogen_Trees.pdf)
- [<b>www <span style='color:#F55AA2'> // </span> INTERFACING WITH PAML</b>](https://biopython.org/wiki/PAML)
- [<b>www <span style='color:#F55AA2'> // </span> WORKING WITH PHYLOGENETIC TREES</b>](https://biopython.org/wiki/PhyloL)
- [<b>www <span style='color:#F55AA2'> // </span> PHYLOGENETIC COOKBOOK</b>](https://biopython.org/wiki/Phylo_cookbook)

### <b><span style='color:#E888BB'> 8.1 |</span> Tree Construction </b>


<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>BIOPYTHON IMPLEMENTATION</span></b></p></div>


- <code>Bio.Phylo.TreeConstruction</code> - Biopython provides several tree construction algorithm implementations in pure python.
- There are a couple of approaches used to construct a <b><span style='color:#F55AA2'>phylogenetic tree</span></b>:
> - <b><span style='color:#F55AA2'>Distance Matrix based</span></b> (<b>implemented in biopython</b>)
> - <b><span style='color:#F55AA2'>Parsimony</span></b> (<b>implemented in biopython</b>)
> - Maximum Likelihood based ( Accessable via <b>external library wrapper</b> )

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>EXTERNAL LIBRARY WRAPPERS</span></b></p></div>


> - <code>Bio.Phylo.PhymlCommandline</code>
> - <code>Bio.Phylo.PAML</code>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>DISTANCE MATRIX BASED APPROACH</span></b></p></div>



In [ ]:
# Recall Alignment
print(alin2,'\n')

In [ ]:
# Recall Alignment
print(alin2,'\n')

''' Calculate Distance Matrix'''
# From raw MSE alignment
# The ‘identity’ model can be used both for DNA and protein sequences

print('Distance Matrix')
calculator = DistanceCalculator('identity') # create calculator
dist_mat = calculator.get_distance(alin2) # get the distance matrix of a given alignment object
print(dist_mat)

In [ ]:
''' Distance Tree Constructor '''
# 'nj' / 'upgma' tree construction approaches

constructor = DistanceTreeConstructor(calculator,'nj') # initialise
tree = constructor.build_tree(alin2) # main algorithm
print(tree)

In [ ]:
# Plot Tree
fig,axes = plt.subplots(1,1,figsize=(12,6))
treep = Phylogeny.from_tree(tree)
Phylo.draw(treep,axes=axes)

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#E888BB'>PARSIMONY BASED APPROACH</span></b></p></div>



In [ ]:
''' Create a starting tree '''

# sub_alin2 =  "/kaggle/working/sub_alin2.fasta"
sub_alin2 = AlignIO.read("/kaggle/working/sub_alin2.fasta", "fasta")

calculator = DistanceCalculator('identity') # create calculator
dist_mat = calculator.get_distance(sub_alin2) # get the distance matrix of a given alignment object
constructor = DistanceTreeConstructor(calculator,'nj') # initialise
tree_0 = constructor.build_tree(sub_alin2) # main algorithm

In [ ]:
# Recall Alignment
print(alin2,'\n')

In [ ]:
from Bio.Phylo.TreeConstruction import ParsimonyScorer, NNITreeSearcher,ParsimonyTreeConstructor

scorer = ParsimonyScorer()
searcher = NNITreeSearcher(scorer)
constructor = ParsimonyTreeConstructor(searcher,tree_0)
pars_tree = constructor.build_tree(alin2)
print(pars_tree)

In [ ]:
# Plot Tree
fig,axes = plt.subplots(1,1,figsize=(12,6))
treep = Phylogeny.from_tree(pars_tree)
Phylo.draw(treep,axes=axes)

### <b><span style='color:#E888BB'> 8.2 |</span> Tree visualisation </b>

In [ ]:
simple = open("/kaggle/working/example.dnd", "w")
simple.write('(((A,B),(C,D),(E,F,G)));')
simple.close()

tree = Phylo.read('/kaggle/working/example.dnd','newick')
print(tree)

In [ ]:
Phylo.draw_ascii(tree)

In [ ]:
Phylo.convert('/kaggle/working/example.dnd','newick',
              '/kaggle/working/tree.xml','phyloxml')
trees = Phylo.parse('/kaggle/working/tree.xml','phyloxml')
for t in trees:
    print(t)

In [ ]:
treep = Phylogeny.from_tree(tree)
Phylo.draw(treep)

In [ ]:
treep.root.color = 'gray'
mrca = treep.common_ancestor({'name':'E'},{'name':'F'})
mrca.color = 'salmon'
treep.clade[0,1].color = 'blue'
Phylo.draw(treep)